<a href="https://colab.research.google.com/github/Abhishek3253/Pyspark/blob/main/assignment07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

In [ ]:

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
df = spark.read.csv(r"/content/Assign07.csv", header=True, inferSchema=True)

In [ ]:
df.show()


+------+----------------+----------+------+-----------+
|userid|         emailid|      date|amount|   discount|
+------+----------------+----------+------+-----------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|
|     5| test5@gmail.com|01.06.2019| 102.5|        2.0|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|
|    11|test11@gmail.com|11-01-2019| 106.7|5.214285714|
|    12|test12@gmail.com|12-01-2019| 107.4|5.535714286|
|    13|test13@gmail.com|13-01-2019| 108.1|5.857142857|
|    14|test14@gmail.com|14-01-2019| 108.8|6.178571429|
|    15|test15@gmail.com|15-01-2019| 109.5|     

In [ ]:
df1 = df.dropna()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
df2 =df.filter(df.date == 'null')

In [ ]:
df2.show()

+------+----------------+----+------+--------+
|userid|         emailid|date|amount|discount|
+------+----------------+----+------+--------+
|    78|test78@gmail.com|null| 258.6|   26.75|
+------+----------------+----+------+--------+



In [ ]:
#UDF function to check date format
from datetime import datetime
def func_date(str1):
  for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy' ,'%m-%d-%Y'):
    try:
      date_object =  datetime.strptime(str1, fmt)
      converted_date = date_object.strftime('%d-%m-%Y')
      return converted_date
    except ValueError:
      pass
  raise ValueError('no valid date format found')

In [ ]:
ab_udf=udf(lambda x:func_date(x))

In [ ]:
df.show(3)

+------+---------------+----------+------+--------+
|userid|        emailid|      date|amount|discount|
+------+---------------+----------+------+--------+
|     1|test1@gmail.com|01-01-2019|   100|     1.0|
|     2|test2@gmail.com|02-01-2019|   100|     1.0|
|     3|test3@gmail.com|03-01-2019|   101|     2.0|
+------+---------------+----------+------+--------+
only showing top 3 rows



In [ ]:
date_udf=df.select(col('userid'),col('emailid'),col('date'),col('amount'),col('discount'),ab_udf(col('date').alias("updated_date")))

In [ ]:
date_udf.show(25)

+------+----------------+----------+------+-----------+------------------------------+
|userid|         emailid|      date|amount|   discount|<lambda>(date AS updated_date)|
+------+----------------+----------+------+-----------+------------------------------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|                    01-01-2019|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|                    02-01-2019|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|                    03-01-2019|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|                    04-01-2019|
|     5| test5@gmail.com|01.06.2019| 102.5|        2.0|                    01-06-2019|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|                    06-01-2019|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|                    07-01-2019|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|                    08-01-2019|
|     9| test9@gmail.com|09-01-2019| 105.3|

In [ ]:
pip install validate_email

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validate-email: filename=validate_email-1.3-py3-none-any.whl size=5482 sha256=d9f22cfd0ca332b3bd3bec6aba132149974e1669edb57e24cfc35ddd38ace7d6
  Stored in directory: /root/.cache/pip/wheels/ff/8f/92/c43287715852eaa75e0d8aa1941c481072b4a82c4f4975074e
Successfully built validate-email


In [ ]:
from validate_email import validate_email
def email_validation(str1):
  is_valid = validate_email(str1)
  if is_valid:
    return str1
  else:
    return "error"

In [ ]:
abi_udf=udf(lambda x:validate_email(x))

In [ ]:
Varify_email=df.select(col("emailid"),abi_udf(col('emailid').alias('varified_result')))

In [ ]:
Varify_email.show()

+----------------+------------------------------------+
|         emailid|<lambda>(emailid AS varified_result)|
+----------------+------------------------------------+
| test1@gmail.com|                                true|
| test2@gmail.com|                                true|
| test3@gmail.com|                                true|
| test4@gmail.com|                                true|
| test5@gmail.com|                                true|
| test6@gmail.com|                                true|
| test7@gmail.com|                                true|
| test8@gmail.com|                                true|
| test9@gmail.com|                                true|
|test10@gmail.com|                                true|
|test11@gmail.com|                                true|
|test12@gmail.com|                                true|
|test13@gmail.com|                                true|
|test14@gmail.com|                                true|
|test15@gmail.com|                              

In [ ]:
df.filter(Varify_email.varified_result=="false").show()

AttributeError: ignored